In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder
import torchvision.transforms as transforms


In [41]:
# Define the transforms for preprocessing the images
transform = transforms.Compose([
    transforms.Resize((227, 227)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])
])

In [42]:
# Set the paths to dataset
train_data_path = '/kaggle/input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/train'
test_data_path = '/kaggle/input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/test'
val_data_path = '/kaggle/input/neu-metal-surface-defects-data/NEU Metal Surface Defects Data/valid'



In [43]:
# Load the dataset
train_dataset = ImageFolder(root=train_data_path, transform=transform)
val_dataset = ImageFolder(root=val_data_path, transform=transform)

In [44]:
# Set the hyperparameters
batch_size = 128
num_epochs = 10
learning_rate = 0.001


In [45]:
# Create data loaders for training and validation
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size, shuffle=False)

In [51]:
# Define the AlexNet model
class AlexNet(nn.Module):
    def __init__(self, num_classes=2):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 96, kernel_size=11, stride=4, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(96, 256, kernel_size=5, padding=2),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
            nn.Conv2d(256, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 384, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(384, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(256 * 6 * 6, 4096),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(4096, 4096),
            nn.ReLU(inplace=True),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [52]:
# Initialize the model
model = AlexNet(num_classes=len(train_dataset.classes))

In [55]:
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [60]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i + 1) % 10 == 0:
            print(f'Epoch [{epoch+1}/{num_epochs}], Step [{i+1}/{total_step}], Loss: {loss.item()}')

Epoch [1/10], Step [10/13], Loss: 1.7743934392929077
Epoch [2/10], Step [10/13], Loss: 1.616544485092163
Epoch [3/10], Step [10/13], Loss: 1.4380099773406982
Epoch [4/10], Step [10/13], Loss: 1.5284479856491089
Epoch [5/10], Step [10/13], Loss: 1.1930261850357056
Epoch [6/10], Step [10/13], Loss: 1.124619960784912
Epoch [7/10], Step [10/13], Loss: 0.7764813899993896
Epoch [8/10], Step [10/13], Loss: 0.6685056090354919
Epoch [9/10], Step [10/13], Loss: 0.6146302819252014
Epoch [10/10], Step [10/13], Loss: 0.6494712233543396


In [67]:
# Evaluate the model

model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    
    for images, labels in val_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
    accuracy = 100 * correct / total
    print(f'Accuracy of the model on the validation set: {accuracy}%')
  

Accuracy of the model on the validation set: 87.5%
